In [1]:
# use packages
using MKL
# using Plots
using Gridap, GridapGmsh, Gridap.Geometry, Gridap.Fields

#Pardiso solver is much faster
# using GridapPardiso
# using SparseMatricesCSR

# For image generation.
#using Images, ImageView,ImageFiltering
using ImageFiltering
# using BenchmarkTools

In [2]:
# constant
# Material property
function lame_parameters(E,ν)
    λ = (E*ν)/((1+ν)*(1-2*ν))
    μ = E/(2*(1+ν))
    (λ, μ)
end
  
#Silicon 
const E_Si = 160.0e9
const ν_Si = 0.27
const (λ_Si, μ_Si) = lame_parameters(E_Si, ν_Si) 
const rho_Si = 2329.0     # kg/m^3
σ_Si(ε) = λ_Si*tr(ε)*one(ε) + 2*μ_Si*ε

# #Vacuum 
# const E_va = 0.0
# const ν_va = 0.0
# const (λ_va, μ_va) = lame_parameters(E_va, ν_va) 
# const rho_va = 0.0    # kg/m^3
# σ_va(ε) = λ_va*tr(ε)*one(ε) + 2*μ_va*ε
  
#PDMS
const E_PDMS = 1.0e6
const ν_PDMS = 0.45
const (λ_PDMS, μ_PDMS) = lame_parameters(E_PDMS, ν_PDMS)
const rho_PDMS = 790.0  #kg/m3
σ_PDMS(ε) = λ_PDMS*tr(ε)*one(ε) + 2*μ_PDMS*ε

# frequency
freq = 1000e6
omega = 2*π*  (freq)
δ=1e-6
L0 = 6 * δ # Length of device
H0 = 4 * δ # Width of device
T0 = 0.2 * δ # Height of device
d_pmlx = 1 * δ  # Thickness of the PML
d_pmly = 0.5 * δ  # Thickness of the PML
d_pmlz = 0.05 * δ  # Thickness of the PML


5.0e-8

In [3]:
# user-defined functions
import Base:real
function real(a :: VectorValue{D, T}) where {D, T<:Number}
    return VectorValue([real(a[i]) for i in 1:D])
end

import Base:imag
function imag(a :: VectorValue{D, T}) where {D<:Int, T<:Number}
    return VectorValue([imag(a[i]) for i in 1:D])
end

import Base:abs
function abs(a :: VectorValue{D, T}) where {D<:Int, T<:Number}
    return VectorValue([abs(a[i]) for i in 1:D])
end



abs (generic function with 20 methods)

In [4]:
# load meshes and view the models
model = GmshDiscreteModel("E:/Work/Postdoc/Quantumphononics/Test/cantilever_small_20231103.msh")
writevtk(model,"cant_model") # write the model to vtk


Info    : Reading 'E:/Work/Postdoc/Quantumphononics/Test/cantilever_small_20231103.msh'...
Info    : 99 entities
Info    : 2490 nodes
Info    : 9390 elements
Info    : Done reading 'E:/Work/Postdoc/Quantumphononics/Test/cantilever_small_20231103.msh'


4-element Vector{Vector{String}}:
 ["cant_model_0.vtu"]
 ["cant_model_1.vtu"]
 ["cant_model_2.vtu"]
 ["cant_model_3.vtu"]

In [5]:
# Define ref/test space
order = 2 # 2nd order Lagrange interpolation

reffe = ReferenceFE(lagrangian,VectorValue{3,ComplexF64},order) # Finite element basis: 2nd order lagrange interpolation of the funciton space

#Linbo: changed to ComplexF64  for have complex number in displacement representing the phase

# V0 = TestFESpace(model,reffe;
#   conformity=:H1,
#   dirichlet_tags=["clamp",],
#   dirichlet_masks=[(true,true,true),])
V = TestFESpace(model,reffe;
  conformity=:H1,)
# g1(x) = VectorValue(0, 0.0, 0.0001) # Dirichlet boundary condition: the third component of the displacement vector is 0.0001
# g1(x) = VectorValue(0, 0.0, 0.0) # Dirichlet boundary condition: the third component of the displacement vector is 0.0001
# U = TrialFESpace(V0,[g1]) # solution function space: Dirichlet: u=g1

U = V # solution function space: Dirichlet: u=g1

UnconstrainedFESpace()

In [6]:
# Define integral region
degree = 2*order # Q: 2 * order of interpolation 
Ω = Triangulation(model) # integral space: bulk; mesh: triangular
dΩ = Measure(Ω,degree) # integral element: gauss-like quadrature in each cell

Ω_d = Triangulation(model, tags="domain1")
dΩ_d = Measure(Ω_d, degree)

# Ω_c = Triangulation(model, tags="PortA")
# dΩ_c = Measure(Ω_c, degree)
#neumanntags = ["clamp", "PortA", "PortB"]
#Γ = BoundaryTriangulation(model,tags=neumanntags)
#Γ = BoundaryTriangulation(model)
#dΓ = Measure(Γ,degree)

GenericMeasure()

In [7]:
p_reffe = ReferenceFE(lagrangian, Float64, order)
Q = TestFESpace(Ω_d, p_reffe, vector_type = Vector{Float64})
P = Q
np = num_free_dofs(P) # Number of cells in design region (number of design parameters)

# pf_reffe = ReferenceFE(lagrangian, Float64, 1)
# Qf = TestFESpace(Ω_d, pf_reffe, vector_type = Vector{Float64})
# Pf = Qf
# fem_params = (; V, U, Q, P, np, Ω, Ω_d, Ω_c, dΩ, dΩ_d, dΩ_c)
fem_params = (; V, U, Q, P ,np, Ω, Ω_d,  dΩ, dΩ_d)


(V = UnconstrainedFESpace(), U = UnconstrainedFESpace(), Q = UnconstrainedFESpace(), P = UnconstrainedFESpace(), np = 16020, Ω = BodyFittedTriangulation(), Ω_d = BodyFittedTriangulation(), dΩ = GenericMeasure(), dΩ_d = GenericMeasure())

In [8]:
# PML in 3D
# Parameters
α = 20 # absorption coefficient

L1 = (d_pmlx ,d_pmly ,d_pmlz) # Corrected size
L2 = (L0-d_pmlx ,H0-d_pmly ,T0-d_pmlz) # Corrected size
d_pml = (d_pmlx ,d_pmly ,d_pmlz)

function s_PML(x, α, L1, L2, d_pml)
    u = abs.(Tuple(x))  # get the depth into PML
    return @. ifelse(
        u < L1, 
        1 + (1im*α)*((L1 - u)/d_pml)^2,
        ifelse(
            u > L2, 
            1 + (1im*α)*((u - L2)/d_pml)^2, 
            1.0+0im
        )
    )
end


struct Λ<:Function
    α::Float64
    L1::NTuple{3,Float64}
    L2::NTuple{3,Float64}  # Here we increased the dimension to 3
    d_pml::NTuple{3,Float64}
end

function (Λf::Λ)(x)
    s_x,s_y,s_z = s_PML(x,Λf.α,Λf.L1,Λf.L2,Λf.d_pml)  # Here we added s_z for the z-axis
    return Gridap.TensorValues.SymTensorValue{3, ComplexF64, 6}(
        1/s_x, 0.0, 0.0, 
        1/s_y, 0.0, 
        1/s_z
    )  # Here we added 1/s_z for the z-axis
end
# function (Λf::Λ)(x)
#     s_x,s_y,s_z = s_PML(x,Λf.α,Λf.L1,Λf.L2,Λf.d_pml)  # Here we added s_z for the z-axis
#     return VectorValue(1/s_x,1/s_y,1/s_z)
# end
# Fields.∇(Λf::Λ) = x->TensorValue{3,3,ComplexF64}(1/s_x,0,0, 0,1/s_y,0, 0,0,1/s_z) 
Λf = Λ(α,L1,L2,d_pml)
# Λf = Λ(α,d_pmlx,d_pmly,d_pmlz,L0,H0,T0)

(::Λ) (generic function with 1 method)

In [9]:
σ = 5
β = 32.0                    # β∈[1,∞], threshold sharpness
η = 0.5                     # η∈[0,1], threshold center
# function Filter(p0;σ)
#   # Apply a 1D Gaussian filter
#   ker = ImageFiltering.Kernel.gaussian((σ,))
#   pf_vec = imfilter(p0, ker)
#       # Threshold the filtered array
#   # pf_vec[pf_vec .> 0.5] .= 1.0
#   # pf_vec[pf_vec .≤ 0.5] .= 0.0
#   return pf_vec
# end
# Function to create a Gaussian kernel as a plain array
function gaussian_kernel(σ)
  radius = ceil(Int, 3*σ)
  x = -radius:radius
  
  # Calculate the Gaussian function
  G = exp.(-x.^2 / (2*σ^2))
  
  # Normalize the kernel
  G /= sum(G)
  
  return G, radius
end

# Function to apply a 1D Gaussian filter manually without in-place mutations
function Filter(p0; σ)
  ker, radius = gaussian_kernel(σ)
  n = length(p0)
  radius = ceil(Int, 3*σ)
  # Use an array comprehension to avoid in-place mutation
  pf_vec = [begin
              acc = 0.0
              for j = 1:length(ker)
                  # Compute the corresponding index in p0
                  idx = i + j - radius - 1
                  # Check if the index is within bounds
                  if idx >= 1 && idx <= n
                      acc += p0[idx] * ker[j]
                  end
              end
              acc  # This value will be the result of the comprehension for index i
            end for i = 1:n]
  return pf_vec
end
function Threshold(pfh; β, η)
  return ((tanh(β * η) + tanh(β * (pfh - η))) / (tanh(β * η) + tanh(β * (1.0 - η))))
end

Threshold (generic function with 1 method)

In [10]:
function σ_d(ε,p)
    
    # change to Si only for debuging.
    # return λ_Si*tr(ε)*one(ε) + 2*μ_Si*ε

    return (p*λ_Si+(1-p)*λ_PDMS)*tr(ε)*one(ε) + 2*(p*μ_Si+(1-p)*μ_PDMS)*ε
  
end

function rho_d(p)

    return p*rho_Si+(1-p)*rho_PDMS

end


rho_d (generic function with 1 method)

In [11]:
using LinearAlgebra
a_design(u,v,pfh) = ( -(Λf.⋅ε(v)) ⊙ (σ_d∘(Λf.⋅ε(u),pfh)) + ((p -> rho_d(p)) ∘ pfh)*omega*omega*u⊙v )
function MatrixA(pfh; fem_params)
    A_mat = assemble_matrix(fem_params.U, fem_params.V) do u, v
         ∫(a_design(u, v, pfh))fem_params.dΩ_d
    end
    return lu(A_mat)
end

MatrixA (generic function with 1 method)

In [12]:
p0 = rand(Float64,fem_params.np)  # Here we make p=0 everywhere just for illustration purpose
# pf_vec = Filter(p0;σ)
pf_vec = p0
pfh = FEFunction(fem_params.P, pf_vec)
pth = (pf -> Threshold(pf; β, η)) ∘ pfh
A_mat = MatrixA(pth; fem_params)
x₀=1.1δ #this should the x coordinate of the center of the cantilever.
f = x -> VectorValue(0.0,0.0,10.0^24/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))
b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
u_vec = A_mat \ b_vec
uh = FEFunction(fem_params.U, u_vec)

SingleFieldFEFunction():
 num_cells: 9270
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 6804373554516831602

In [13]:
function MatrixOf(fem_params)
    x0 = VectorValue(3δ,-2δ,0.1δ)  # Position of the field to be optimized
    return assemble_matrix(fem_params.U, fem_params.V) do u, v
        ∫((x->(1/(2*π)*exp(-norm(x - x0)^2 / 2 / δ^2))) * (ε(u) ⊙ ε(v)) )fem_params.dΩ
    end
end
# O=MatrixOf(fem_params)
# println(maximum(abs.(O)))

MatrixOf (generic function with 1 method)

In [14]:
using ChainRulesCore, Zygote
import ChainRulesCore: rrule
NO_FIELDS = ZeroTangent()

ZeroTangent()

In [15]:


function dσ_dpt(ε)
    

    return (λ_Si-λ_PDMS)*tr(ε)*one(ε) + 2*(μ_Si-μ_PDMS)*ε
  
end

function da_drho(u,v)

    return omega*omega*u⊙v

end

drho_dpt = rho_Si-rho_PDMS

function da_dσ(ε)

    return -ε

end
Dptdpf(pf, β, η) = β * (1.0 - tanh(β * (pf - η))^2) / (tanh(β * η) + tanh(β * (1.0 - η)))

# DAdpf(u,v,pfh;β, η) = (-(Λf.⋅ε(v)) ⊙ (dσ_dpt∘(Λf.⋅ε(u))) + (drho_dpt*da_drho(u,v))) * ((p -> Dξdpf(p, β, η)) ∘ pfh)
DAdpf(u,v,pfh;β, η) = (da_dσ∘(Λf.⋅ε(v)) ⊙ (dσ_dpt∘(Λf.⋅ε(u))) + (drho_dpt*da_drho(u,v))) * ((p -> Dptdpf(p, β, η)) ∘ pfh)

DAdpf (generic function with 1 method)

In [16]:
function gf_pf(pf_vec; f,β, η, fem_params)
    pfh = FEFunction(fem_params.P, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth; fem_params)
    b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
    u_vec = A_mat \ b_vec

    O_mat = MatrixOf(fem_params)
    real(u_vec' * O_mat * u_vec)
end

function rrule(::typeof(gf_pf), pf_vec; f,β, η, fem_params)
    function U_pullback(dgdg)
      NO_FIELDS, dgdg * Dgfdpf(pf_vec; f,β, η,  fem_params)
    end
    gf_pf(pf_vec; f,β, η,  fem_params), U_pullback
end

function Dgfdpf(pf_vec; f,β, η, fem_params)
    pfh = FEFunction(fem_params.P, pf_vec)
    pth = (pf -> Threshold(pf; β, η)) ∘ pfh
    A_mat = MatrixA(pth;  fem_params)
    b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
    u_vec = A_mat \ b_vec
    O_mat = MatrixOf(fem_params)

    uh = FEFunction(fem_params.U, u_vec)
    w_vec =  A_mat' \ (O_mat * u_vec)
    wconjh = FEFunction(fem_params.U, conj(w_vec))

    l_temp(dp) = ∫(real(-2 * DAdpf(uh, wconjh, pfh;  β, η)) * dp)fem_params.dΩ_d
    dgfdpf = assemble_vector(l_temp, fem_params.P)
    return dgfdpf
end

Dgfdpf (generic function with 1 method)

In [26]:

function pf_p0(p0; σ)
    pf_vec = Filter(p0; σ)
    # pf_vec = p0
    pf_vec
end
function rrule(::typeof(pf_p0), p0; σ)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf,p0; σ,fem_params)
  end
  pf_p0(p0; σ), pf_pullback
end
function gaussian_derivative_matrix(σ, p0)
  ker, radius = gaussian_kernel(σ)
  kernel_length = length(ker)
  length_p0 = length(p0)
  derivative_matrix = zeros(length_p0, length_p0)
  for i = 1:length_p0
      for j = 1:kernel_length
          idx = i + j - radius - 1
          if idx >= 1 && idx <= length_p0
              derivative_matrix[i, idx] = ker[j]
          end
      end
  end
  
  return derivative_matrix
end

function Dgdp(dgdpf,p0; σ,fem_params)
  Af = gaussian_derivative_matrix(σ, p0)
  # Af = Diagonal(ones(Int,length(p0)))
  wvec = Af' \ dgdpf
  wh = FEFunction(fem_params.P, wvec)
  l_temp(dp) = ∫(wh * dp)fem_params.dΩ_d
  return assemble_vector(l_temp, fem_params.P)
end

Dgdp (generic function with 1 method)

In [29]:
function gf_p(p0::Vector;  σ,f,β, η,fem_params)
    # pf_vec = pf_p0(p0; σ)
    pf_vec = p0
    gf_pf(pf_vec; f,β, η, fem_params)
end

function gf_p(p0::Vector, grad::Vector;  σ,f,β, η,fem_params)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> gf_p(p;  σ,f,β, η,fem_params), p0)
        grad[:] = dgdp
    end
    gvalue = gf_p(p0::Vector; σ,f,β, η,fem_params)
    open("gvalue.txt", "a") do io
        write(io, "$gvalue \n")
    end
    gvalue
end

gf_p (generic function with 2 methods)

In [30]:
p0 = rand(fem_params.np)
δp = rand(fem_params.np)*1.0e-8
grad = zeros(fem_params.np)

g0 = gf_p(p0, grad;  σ,f, β, η,fem_params)
g1 = gf_p(p0+δp, []; σ,f, β, η,fem_params)
g1-g0, grad'*δp

(2.254665600101818e-12, 2.2546337924839496e-12)

In [ ]:
using NLopt

function gf_p_optimize(p_init; TOL = 1e-4, MAX_ITER = 500, f, fem_params)
    ##################### Optimize #################
    opt = Opt(:LD_MMA, fem_params.np)
    opt.lower_bounds = 0
    opt.upper_bounds = 1
    opt.ftol_rel = TOL
    opt.maxeval = MAX_ITER
    opt.max_objective = (p0, grad) -> gf_p(p0, grad; f, fem_params)

    (g_opt, p_opt, ret) = optimize(opt, p_init)
    @show numevals = opt.numevals # the number of function evaluations
    return g_opt, p_opt
end

p_opt = fill(0.4, fem_params.np)   # Initial guess


g_opt = 0
TOL = 1e-8
MAX_ITER = 100
g_opt, p_temp_opt = gf_p_optimize(p_opt; TOL, MAX_ITER, f, fem_params)
global p_opt = p_temp_opt

@show g_opt

In [ ]:
using CairoMakie, GridapMakie
p0 = p_opt

p_vec = p0
pfh = FEFunction(fem_params.P, p_vec)

A_mat = MatrixA(pfh; fem_params)
f = x -> VectorValue(0.0,0.0,10.0^24/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))
b_vec = assemble_vector(v->(∫(v⋅f)fem_params.dΩ), fem_params.V)
u_vec = A_mat \ b_vec
uh = FEFunction(fem_params.U, u_vec)

fig, ax, plt = plot(fem_params.Ω, pfh, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(1)
ax.title = "Design Shape"
rplot = 110 # Region for plot
limits!(ax, 0, 6δ, 0, -4δ)
save("shape.png", fig)

In [ ]:
maxe = 30 # Maximum electric field magnitude compared to the incident plane wave
# u1=abs2(phys_params.n_air^2)
# u2=abs2(phys_params.n_metal^2)

fig, ax, plt = plot(fem_params.Ω, 2*(sqrt∘(abs((conj(ε(uh)) ⋅ ε(uh))))), colormap = :hot, colorrange=(0, maxe))
Colorbar(fig[1,2], plt)
ax.title = "|u|"
ax.aspect = AxisAspect(1)
limits!(ax, 0, 6δ, 0, -4δ)
save("Field.png", fig)

In [ ]:

x₀=1.1δ #this should the x coordinate of the center of the cantilever.

f = x -> VectorValue(0.0,0.0,1/(2π) * exp(-(x[1]-x₀)^2/2/δ^2))


# l(v) = 0
l(v) = ∫( v⋅f )*dΩ
# a(u,v) = ∫( -ε(v) ⊙ (σ_Si∘(ε(u))) + rho_Si*omega*omega*u⊙v )*dΩ

a(u,v) = ∫( -(Λf.⋅ε(v)) ⊙ (σ_Si∘(Λf.⋅ε(u))) + rho_Si*omega*omega*u⊙v )*dΩ
println("assemble 2")
op = AffineFEOperator(a,l,U,V0)

println("solving 2")
#solve
solver = LinearFESolver()
uh = solve(solver, op)

println("writing solution 1")
writevtk(Ω,"20230922_pureSi_shift_no_PML",cellfields=["uhR"=>real(uh), "rho"=>rho_Si])

println("complete.")

In [ ]:
g = zeros(Float64, 11)

for c in 1:1:11
    g[c] = real(uh(VectorValue(10δ,-2δ,(c-1)*δ/10)))[3]
end

p = plot((0:1:10)*δ/10, g, label="With PML", xlims=(0,10*δ/10), ylims=(-4e-24,4e-24),
         xlabel="z/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:5e-6,yticks=-4e-24:2e-24:4e-24)

# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [1e-6, 3e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)

In [ ]:
g = zeros(Float64, 41)

for c in 1:1:41
    g[c] = real(uh(VectorValue(10δ,-(c-1)*δ/10,0.5δ)))[3]
end

# p = plot((0:1:40)*δ/10, g, label="With PML", xlims=(0,40*δ/10), ylims=(-4e-24,4e-24),
#          xlabel="y/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:5e-6,yticks=-4e-24:2e-24:4e-24)
p = plot((0:1:40)*δ/10, g, label="With PML", xlims=(0,40*δ/10),ylims=(0,2e-24),
         xlabel="y/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:1e-6:4e-6)
# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [1e-6, 3e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)

In [ ]:


g = zeros(Float64, 221)

for c in 1:1:221
    g[c] = abs(real(uh(VectorValue((c-1)*δ/10,-2δ,0.5δ)))[3])
end

# p = plot((0:1:200)*δ/10, g, label="With PML", xlims=(0,200*δ/10), ylims=(-4e-24,4e-24),
#          xlabel="x/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:5e-6:20e-6,yticks=-4e-24:2e-24:4e-24)
p = plot((0:1:220)*δ/10, g, label="With PML", xlims=(0,220*δ/10), 
         xlabel="x/m", ylabel="real(uz)", framestyle=:box,seriestype=:line, grid=true,xticks=0:2e-6:22e-6,yticks=0:0.4e-24:2e-24)
# Add vertical lines at x = 2e-6 and 18e-6
vline!(p, [2e-6, 20e-6], color=:red, linestyle=:dash,label="")

# Display the updated plot
display(p)


In [ ]:
# Scalar value used for inverse design

result_freq = Vector{Float64}()
result_A = Vector{Float64}()
result_B = Vector{Float64}()

A = BoundaryTriangulation(model, tags="PortA") # Boundary triangulation assumes neumann boundary
dA = CellQuadrature(A, degree)
int1 = sum(integrate(uh, dA ))
int2 = sum(integrate(1,dA))
response_A = int1/int2 /0.001 # 
resZ_A = abs(response_A[3])
append!(result_A, resZ_A)



B = BoundaryTriangulation(model, tags="PortB")
dB = CellQuadrature(B, degree)
int1 = sum(integrate(uh, dB ))
int2 = sum(integrate(1,dB))
response_B = int1/int2 /0.001
resZ_B = abs(response_B[3])
append!(result_B, resZ_B)


println("$freq, $resZ_A, $resZ_B")